<a href="https://colab.research.google.com/github/noproblama25/inflation/blob/main/Inflation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Investigating inflation trends in the Netherlands

# 1. Inflation (target)


### 1.1 Datasets

Source (ECB): https://data.ecb.europa.eu/data/datasets/ICP/ICP.M.BE.N.000000.4.ANR

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import matplotlib.dates as mdates

In [ ]:
df_de = pd.read_csv("/content/inflatie/ECB Data Portal_20251029152601_DE.csv")
df_nl = pd.read_csv("/content/inflatie/ECB Data Portal_20251029152915_NL.csv")
df_eu = pd.read_csv("/content/inflatie/ECB Data Portal_20251029153138_EU.csv")
df_be = pd.read_csv("/content/inflatie/ECB Data Portal_20251029153237_BE.csv")

In [ ]:
df_de.columns

In [ ]:
# Determine the name of the third column (index 2 since indexing starts at 0)
current_columns_de = df_de.columns
current_columns_nl = df_nl.columns
current_columns_eu = df_eu.columns
current_columns_be = df_be.columns

# Rename the column DE

column_to_rename_de = current_columns_de[2]
new_column_name_de = "Inflation_DE"  # Replace with your desired new name
df_de = df_de.rename(columns={column_to_rename_de: new_column_name_de})

# Rename the column NL

column_to_rename_nl = current_columns_nl[2]
new_column_name_nl = "Inflation_NL"  # Replace with your desired new name
df_nl = df_nl.rename(columns={column_to_rename_nl: new_column_name_nl})

# Rename the column BE
column_to_rename_be = current_columns_be[2]
new_column_name_be = "Inflation_BE"  # Replace with your desired new name
df_be = df_be.rename(columns={column_to_rename_be: new_column_name_be})

# Rename the column EU
column_to_rename_eu = current_columns_eu[2]
new_column_name_eu = "Inflation_EU"  # Replace with your desired new name
df_eu = df_eu.rename(columns={column_to_rename_eu: new_column_name_eu})


In [ ]:
# adding country code label

df_de['Country_Code'] = 'DE'
df_nl['Country_Code'] = 'NL'
df_be['Country_Code'] = 'BE'
df_eu['Country_Code'] = 'EU'

In [ ]:
# combine all dataframes into a single dataframe by concatenating rows
df_all = pd.concat([df_de, df_nl, df_be, df_eu], axis=0, ignore_index=True)

In [ ]:
# Melt the DataFrame
df_melted = pd.melt(df_all, id_vars=['DATE', 'TIME PERIOD', 'Country_Code'],
                    value_vars=['Inflation_DE', 'Inflation_NL', 'Inflation_BE', 'Inflation_EU'],
                    var_name='Country_Inflation_Column', value_name='Inflation')

# Drop the original 'Country_Inflation_Column' as Country_Code is already available
df_melted = df_melted.drop(columns=['Country_Inflation_Column'])

# convert "DATE" column to date format
df_melted['DATE'] = pd.to_datetime(df_melted['DATE'])

# clean all recordsw with blank/null values in Inflation column
df_melted = df_melted.dropna(subset=['Inflation'])

In [ ]:
import matplotlib.dates as mdates

plt.figure(figsize=(12, 6))
sns.lineplot(data=df_melted, x='DATE', y='Inflation', hue='Country_Code')
plt.title('Inflation Trends Over Time by Country')
plt.xlabel('Date')
plt.ylabel('Inflation (%)')
plt.grid(False)

# Set the x-axis major locator and formatter
ax = plt.gca()
ax.xaxis.set_major_locator(mdates.YearLocator(1)) # Show a tick every year
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y')) # Format the date as YYYY-MM-DD

# Set the x-axis limits to match the data range
ax.set_xlim(df_melted['DATE'].min(), df_melted['DATE'].max())

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

### Observations: Nethelrands had increased inflation in three periods:

2001-2003  
2013-2014  
2019-2022 mid year  
2024-2025 sep  


### 1.2 Isolating periods of elevated inflation

In [ ]:
# changing DATE format
df_de['DATE'] = pd.to_datetime(df_de['DATE'])
df_be['DATE'] = pd.to_datetime(df_be['DATE'])
df_nl['DATE'] = pd.to_datetime(df_nl['DATE'])
df_eu['DATE'] = pd.to_datetime(df_eu['DATE'])

# setting index
df_de.set_index('DATE', inplace=True)
df_be.set_index('DATE', inplace=True)
df_nl.set_index('DATE', inplace=True)
df_eu.set_index('DATE', inplace=True)

# combine all dataframes into a single dataframe using DATE column as index
df_all = pd.concat([df_de, df_nl, df_be, df_eu], axis=1, ignore_index=False,join='inner')

# Drop all records where DATE <= 1996-12-31
df_all = df_all.reset_index()
df_all = df_all[df_all['DATE'] > '1996-12-31']

# drop columns
df_all = df_all.drop(columns=['TIME PERIOD','Country_Code','TIME PERIOD','Country_Code','TIME PERIOD','Country_Code'])


Isolating periods where NL inflation was higher than DE and BE

In [ ]:
# checking the inflation in NL is higher than average inflaiton of the two neighboring countries (DE, BE)
# using average of the two countries to reduce the noise and consider only significant outliers

df_all['Inf_elev'] = (df_all['Inflation_NL'] > (df_all['Inflation_BE'] + df_all['Inflation_DE'])/2)

In [ ]:
df_all